In [1]:
import numpy as np
import scipy.sparse as sp
import cupy as cp
import cupyx.scipy.sparse as cpsp
import time
from sklearn.datasets import load_svmlight_file

In [2]:
# # Make sure you have GPU available
# print("GPU available:", cp.is_available())

# # Function to create a random sparse matrix
# def create_sparse_matrix(n, density):
#     return sp.random(n, n, density=density, format='csr')

# Function to run CPU sparse matrix-vector product
def cpu_spmv(matrix, vector):
    return matrix @ vector

# Function to run GPU sparse matrix-vector product
def gpu_spmv(matrix, vector):
    return matrix @ vector

# # Test parameters
# n = 100000  # matrix size
# density = 0.01  # sparsity
# num_runs = 10  # number of runs for averaging

# # Create sparse matrix and vector
# cpu_matrix = create_sparse_matrix(n, density)
# cpu_vector = np.random.rand(n)

# # Transfer to GPU
# gpu_matrix = cpsp.csr_matrix(cpu_matrix)
# gpu_vector = cp.array(cpu_vector)

In [3]:
# # Warm-up run
# _ = cpu_spmv(cpu_matrix, cpu_vector)
# _ = gpu_spmv(gpu_matrix, gpu_vector)
# cp.cuda.Stream.null.synchronize()

# # CPU timing
# cpu_times = []
# for _ in range(num_runs):
#     start = time.time()
#     _ = cpu_spmv(cpu_matrix, cpu_vector)
#     cpu_times.append(time.time() - start)

# # GPU timing
# gpu_times = []
# for _ in range(num_runs):
#     start = time.time()
#     _ = gpu_spmv(gpu_matrix, gpu_vector)
#     cp.cuda.Stream.null.synchronize()
#     gpu_times.append(time.time() - start)

# # Print results
# print(f"Matrix size: {n}x{n}, Density: {density}")
# print(f"CPU average time: {np.mean(cpu_times):.6f} seconds")
# print(f"GPU average time: {np.mean(gpu_times):.6f} seconds")
# print(f"Speedup: {np.mean(cpu_times) / np.mean(gpu_times):.2f}x")

In [4]:
# X, _ = load_svmlight_file("url_combined_normalized")
X = sp.load_npz("yelp_train.npz").astype(np.float64)
X_gpu = cp.sparse.csr_matrix(X)

In [5]:
v_cpu = np.random.rand(X.shape[1])
v_gpu = cp.array(v_cpu)

# Warm-up run
_ = cpu_spmv(X, v_cpu)
_ = gpu_spmv(X_gpu, v_gpu)
cp.cuda.Stream.null.synchronize()

# CPU timing
cpu_times = []
for _ in range(10):
    start = time.time()
    _ = cpu_spmv(X, v_cpu)
    cpu_times.append(time.time() - start)

# GPU timing
gpu_times = []
for _ in range(10):
    start = time.time()
    _ = gpu_spmv(X_gpu, v_gpu)
    cp.cuda.Stream.null.synchronize()
    gpu_times.append(time.time() - start)

In [6]:
# Print results
print(f"Matrix size: {X.shape[0]}x{X.shape[1]}, Density: {X.nnz / (X.shape[0] * X.shape[1])}")
print(f"CPU average time: {np.mean(cpu_times):.6f} seconds")
print(f"GPU average time: {np.mean(gpu_times):.6f} seconds")
print(f"Speedup: {np.mean(cpu_times) / np.mean(gpu_times):.2f}x")

Matrix size: 5038676x2084724, Density: 7.318352123063983e-05
CPU average time: 0.786778 seconds
GPU average time: 0.022826 seconds
Speedup: 34.47x
